Functions

In [275]:
import re
import numpy as np

indices = {}


def load(name):
    """
    Load the data

    params:
    name:   file directory and name
    """
    arr = []
    with open(name) as f:
        for i in f:
            x = [j.strip() for j in re.split(',|\[|\]', i)]
            xtmp = []
            xtmp.append(int(x[1]))
            xtmp.append(int(x[2]))
            xtmp2 = []
            xtmp2.append(int(x[4]))
            xtmp2.append(int(x[5]))
            arr.append(xtmp)
            arr.append(xtmp2)

    return arr


# textArr.sort(key=takeSecond)
def takeThird(arr):
    """
    Used as a key to return the 2nd element in an array

    params:
    arr:    input array
    """
    return arr[2]


def dist(a, b):
    return np.sqrt(np.sum((a-b)**2))


def dist_2(x, y):
    x1 = x[0]
    x2 = x[1]
    y1 = y[0]
    y2 = y[1]
    return ((x1 - y1)**2 + (x2 - y2)**2)**0.5
    

def pairwise_distances(myArray):
    # Set negative values to zero
    myArray[myArray < 0] = 0
    d = np.sqrt(np.sum((myArray[:, np.newaxis, :] - myArray[np.newaxis, :, :])**2, axis=-1))
    return d


def buildSet(arr):
    my_set = set()
    my_keys = []
    c = 0
    for i in arr:
        x = (i[0], i[1])
        if x not in my_set:
            my_set.add((i[0], i[1]))
            my_keys.append(c)
            c += 1
    l = list(my_set)
    return l, my_keys


def create_map(lst):
    indices = {}
    for i, x in enumerate(lst):
        indices[x] = i
    return indices


def find_index(element):
    return indices[element]

KD Tree

In [276]:
# Chat GPT helped me construct this
class Node:
    def __init__(self, data=None, split=None, left=None, right=None):
        self.data = data
        self.split = split
        self.left = left
        self.right = right

class KDTree:
    def __init__(self, points):
        self.root = self.build_kdtree(points)
    
    def build_kdtree(self, points, depth=0):
        n = len(points)
        if n <= 0:
            return None
        
        k = len(points[0])
        axis = depth % k
        sorted_points = sorted(points, key=lambda point: point[axis])
        
        mid = n // 2
        return Node(
            data=sorted_points[mid],
            split=axis,
            left=self.build_kdtree(sorted_points[:mid], depth+1),
            right=self.build_kdtree(sorted_points[mid+1:], depth+1)
        )
    
    def search_knn(self, query, k=1):
        knn = []
        self.search_knn_helper(query, self.root, knn, k)
        return knn

    def search_knn_helper(self, query, node, knn, k):
        if node is None:
            return
        
        axis = node.split
        if query[axis] < node.data[axis]:
            self.search_knn_helper(query, node.left, knn, k)
            further_child = node.right
        else:
            self.search_knn_helper(query, node.right, knn, k)
            further_child = node.left
        
        if node.data != query:  # skip nodes whose point is the same as the query point
            knn.append(node.data)
            knn.sort(key=lambda point: np.linalg.norm(np.array(point) - query))
            while len(knn) > k:
                knn.pop()
            
            if further_child is not None and \
            abs(query[axis] - node.data[axis]) < np.linalg.norm(np.array(knn[-1]) - query):
                self.search_knn_helper(query, further_child, knn, k)



Class

In [277]:
# Credit to: https://www.programiz.com/dsa/kruskal-algorithm
class Graph:
    def __init__(self, vertices, edge):
        self.V = vertices
        self.graph = edge

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])

    def get(self):
        return self.graph

    # Search function
    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    def apply_union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    #  Applying Kruskal algorithm
    def kruskal_algo(self, initial_edges):
        result = initial_edges[:]
        i, e = 0, len(initial_edges)
        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = []
        rank = []
        total = 0
        for node in range(self.V):
            parent.append(node)
            rank.append(0)
        for k in result:
            u = k[0]
            v = k[1]
            w = k[2]
            x = self.find(parent, u)
            y = self.find(parent, v)
            self.apply_union(parent, rank, x, y)
        while e < self.V - 1:
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.apply_union(parent, rank, x, y)
                total += w
        # for u, v, weight in result:
            # print("%d - %d: %d" % (u, v, weight))
        print("Total = ", total)

Load

In [278]:
arr = load("./graf18.txt")

Keys

In [279]:
myList, myKeys = buildSet(arr)
num_vertices = len(myList)
num_edges = int((num_vertices *  (num_vertices - 1)) / 2)

Indices

In [280]:
indices = create_map(myList)

Initial Edges

In [282]:
initial_edges = []
for i in range(0, len(arr), 2):
    initial_edges.append([find_index((arr[i][0], arr[i][1])), find_index((arr[i + 1][0], arr[i + 1][1])), dist_2(arr[i], arr[i + 1])])


Make Graph

In [283]:
g = Graph(num_vertices, initial_edges)
g.get()

[[6, 11, 1.0],
 [11, 4, 2.0],
 [16, 7, 1.0],
 [8, 14, 1.0],
 [14, 5, 1.0],
 [0, 9, 2.0],
 [9, 3, 2.0],
 [3, 12, 1.0],
 [13, 17, 2.0],
 [1, 10, 1.0],
 [15, 2, 3.0]]

Tree

In [200]:
kd = KDTree(myList)

Find new edges

Algorithm

In [202]:
# g.kruskal_algo(initial_edges)

TypeError: list indices must be integers or slices, not numpy.float64